**CMP585 - Structural Bioinformatics**

Cristian Lopes

*Assignment 2*

In [1]:
import pandas as pd
import numpy as np
import glob

# Load Data

In [2]:
def read_molecule(path):
    atoms = []
    with open(path, 'r') as f:
        for line in f:
            if line.startswith('ATOM'):
                atoms.append({'atom_name': line[12:16].strip(), 
                             'residue_name': line[17:20].strip(),
                             'residue_seq_number': line[22:26].strip(),
                             'x': float(line[30:38].strip()), 
                             'y': float(line[38:46].strip()),
                             'z': float(line[46:54].strip())})
    return pd.DataFrame(atoms)

In [3]:
molecule = read_molecule('./data/1crn.pdb')

In [4]:
molecule.head(20)

,atom_name,residue_name,residue_seq_number,x,y,z
0,N,THR,1,17.047,14.099,3.625
1,CA,THR,1,16.967,12.784,4.338
2,C,THR,1,15.685,12.755,5.133
3,O,THR,1,15.268,13.825,5.594
4,CB,THR,1,18.170,12.703,5.337
5,OG1,THR,1,19.334,12.829,4.463
6,CG2,THR,1,18.150,11.546,6.304
7,N,THR,2,15.115,11.555,5.265
8,CA,THR,2,13.856,11.469,6.066
9,C,THR,2,14.164,10.785,7.379


# Q1) Primary to Tertiary Structure

## Symbol of each amino acid

First of all, we define the symbol of each amino acid.

In [5]:
symbol_to_residue_name = {
    'T': 'THR', 'C': 'CYS', 'P': 'PRO', 'S': 'SER', 'I': 'ILE',
    'V': 'VAL', 'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'F': 'PHE',
    'L': 'LEU', 'G': 'GLY', 'E': 'GLU', 'D': 'ASP', 'Y': 'TYR'
}

## Load Amino Acids 

Then, we load the amino acids from the Protein Data Bank.

In [6]:
AMINOACID_PATH = './data/amino_acids/'
aminoacids = pd.DataFrame()

for f in glob.glob(AMINOACID_PATH + "*.pdb"):
    aminoacids = aminoacids.append(read_molecule(f))

In [7]:
aminoacids.head(20)

,atom_name,residue_name,residue_seq_number,x,y,z
0,N,ALA,1,0.039,-0.028,0.000
1,CA,ALA,1,1.499,-0.043,0.000
2,C,ALA,1,2.055,1.361,0.000
3,O,ALA,1,1.321,2.356,0.011
4,CB,ALA,1,1.956,-0.866,-1.217
5,1H,ALA,1,-0.524,0.894,0.000
6,2H,ALA,1,-0.543,-0.938,0.000
7,HA,ALA,1,1.847,-0.534,0.928
8,1HB,ALA,1,3.058,-0.939,-1.274
9,2HB,ALA,1,1.571,-1.903,-1.181


Right now, we have a dictionnary of amino acids containing the position of each atom inside the amino acid.

## Primary structure to tertiary structure

### Translation and Rotation

Next step is to replace each symbol of the primary structure by its corresponding residue. First, we define a translation and a rotation function to move and rotate the next amino acid given the position of the previous aminoacid.

In [180]:
def translate(aminoacid, translation):
    aminoacid[['x', 'y', 'z']] = aminoacid[['x', 'y', 'z']] + translation.values.reshape(-1)
    return aminoacid

In [182]:
def translation_to_origin(aminoacid):
    ref_atom = 'N'
    ref_translation = aminoacid[aminoacid.atom_name == ref_atom][['x', 'y', 'z']]
    return ref_translation, translate(aminoacid, -ref_translation)
    

def rotate(aminoacid, angle_x, angle_y, angle_z):  
    Rx = [[1, 0, 0], [0, np.cos(angle_x), -np.sin(angle_x)], [0, np.sin(angle_x), np.cos(angle_x)]]
    Ry = [[np.cos(angle_y), 0, np.sin(angle_y)], [0, 1, 0], [-np.sin(angle_y), 0, np.cos(angle_y)]]
    Rz = [[np.cos(angle_z), -np.sin(angle_z), 0], [np.sin(angle_z), np.cos(angle_z), 0], [0, 0, 1]]
    R = np.dot(np.dot(Rx, Ry), Rz)
    
    ref_translation, aminoacid = translation_to_origin(aminoacid)
    aminoacid[['x', 'y', 'z']] = R.dot(aminoacid[['x', 'y', 'z']].values.T).T
    aminoacid = translate(aminoacid, ref_translation)
    
    return aminoacid

### Planar Peptide Bond Constraints

Given the position of  O and C (Carboxyl) from the previous amino acid and the position of N from the next one, we can calculate the necessary translation and rotation in the next amino acid in order to form a planar peptide bond, that means, Omega equals to 180° (considering a distance of 1.33 ångström between C and N and an angle of 123,5° between O-C and C-N as shown in the figure below).

<div style="text-align:center"><img width="600px" src="./images/planar_peptide_bond.png"/></div>
<br>
<br>
<div style="text-align:center">Figure 1:  The average dimensions of the planar peptide group (in angstroms and degrees).</div>
<div style="text-align:center">**Source:** https://www.open.edu/openlearn/science-maths-technology/science/biology/proteins/content-section-1.2)</div>

In [ ]:
def calculate_distance(p1, p2): # distance between Ci and Ni+1 should be 1.33 ångström
    return np.sqrt(np.sum((p1-p2) ** 2))

In [ ]:
def calculate_angle(p1, p2, p3): # angle between Oi-Ci and Ci-Ni+1 should be 123.5°
    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)
    
    p1_ref = p1 - p2
    p3_ref = p3 - p2
    
    cosine_angle = np.dot(p1_ref, p3_ref) / (np.linalg.norm(p1_ref) * np.linalg.norm(p3_ref))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

In [ ]:
def calculate_dihedral_angle(p1, p2, p3, p4): # dihedral angle between Calphai-Ci-Ni+1 and Ci-Ni+1-Calphai+1 should be 180°
    p1, p2, p3, p4 = np.array(p1), np.array(p2), np.array(p3), np.array(p4)
    
    b1 = -1.0 * (p2 - p1)
    b2 = p3 - p2
    b3 = p4 - p3
    
    # normalize b2
    b2 /= np.linalg.norm(b2)
    
    # projections of b1 and b3 onto plane perpendicular to b2
    v = b1 - np.dot(b1, b2) * b2
    w = b3 - np.dot(b3, b2) * b2
    
    # angle between v and w is the torsion angle
    x = np.dot(v, w)
    y = np.dot(np.cross(b2, v), w)
    return np.degrees(np.arctan2(y, x))

### Calculate Next Amino Acid Translation and Rotation

Based on the constraints, we can calculate the translation and rotation of the next amino acid given the previous one.

In [ ]:
def _calculate_next_residue_translation(previous_aminoacid, next_aminoacid):
    pass

def _calculate_next_residue_rotation(previous_aminoacid, next_aminoacid):
    pass

### Generate Tertiary Structure

Once we have a way to calculate the translation and rotation of the next aminoacidd in the chain given the previous one, we can implement the routine to generate the tertiary structure of a protein given the primary structure.

In [ ]:
primary_structure = 'TTCCPSIVARSNFNVCRLPGTPEAICATYTGCIIIPGATCPGDYAN'

In [ ]:
def _infer_tertiary_structure(primary_structure, aminoacids_df):
    protein = pd.DataFrame(data=None, columns=['atom_name', 'residue_name', 'residue_seq_number', 'x', 'y', 'z'])    
    previous_aminoacid = None
    
    for i, r in enumerate(primary_structure):
        aminoacid = aminoacids_df[aminoacids_df.residue_name == symbol_to_residue_name[r]].copy()

        if previous_aminoacid is not None:
            translation = _calculate_next_residue_translation(previous_aminoacid, aminoacid)
            rotation = _calculate_next_residue_rotation(previous_aminoacid, aminoacid)
        
        aminoacid = translate(aminoacid, translation)
        aminoacid = rotate(aminoacid, rotation)
        aminoacid['residue_seq_number'] = i
        
        previous_aminoacid = aminoacid
        protein = protein.append(aminoacid, sort=True, ignore_index=True)
    return protein

In [ ]:
protein = _infer_tertiary_structure(primary_structure, aminoacids)

In [ ]:
protein.head(20)

Done!